In [1]:
from __future__ import division
import os

import numpy as np
from numpy import pi

import pandas as pd
from numpy import sqrt
from scipy import integrate

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def journal():
    '''
    Appears nearly square. I like this best.
    'figure.figsize': [3.1, 2.6]
    
    Use: plt.tight_layout(pad=0.1, w_pad=0.0, h_pad=0.0)
    plt.xlabel('$E_{iso}$ $(erg)$',labelpad=-1)  
    plt.ylabel('$E^*_{pk}$ $(keV)$',labelpad=-2)
    
    '''
    params = {'backend': 'pdf',
              'axes.labelsize':  10,
              'font.size':       10,
              'legend.fontsize': 8,
              'xtick.labelsize': 8,
              'ytick.labelsize': 8,
              'xtick.direction': 'in',
              'ytick.direction': 'in',
              'text.usetex':     True,
              'figure.figsize': [4,3], #[4,3], #[3.1, 2.6], # [7,6]
              'font.family': 'serif',}
    plt.rcParams.update(params)

In [3]:
journal()

#### All luminosity distance functions must pass results in parsecs.

In [4]:
def LumDist_concordance(redshift, H_knot=None, omega_m=None):
    """
    This is the function we use in our work, but different cosmo constants 
    as well as DL units. 
    """
    z           = redshift
    if H_knot is None:
        H_knot      = 71.               # Hubble Constant.  Units: km/s/Mpc
    if omega_m is None:
        omega_m     = 0.27             # Matter Density.
    omega_l     = 1.0 - omega_m
    
    c           = 2.99792458e5      # SPEED OF LIGHT    Units: km/s
    def Aint(z):
        return (1./(sqrt(((1.+z)*(1.+z)*(1. + omega_m * z))-(z*(2.+ z)*omega_l ))))
    AA      = integrate.quad(Aint, 0.0, z)
    DL_Mpc  = (c * (1. + z) / H_knot) * AA[0]
    DL_pc = DL_Mpc * (1E6) # convert Mpc to pc
    return DL_pc


def LumDist_riess(redshift, H_knot=None, omega_m=None):
    """
    Lower order expansion for dark energy term (w). Equation 14 in Riess et al. 2004. 
    w(z) = w0 + w'z
    """
    z           = redshift
    if H_knot is None:
        H_knot      = 71.               # Hubble Constant.  Units: km/s/Mpc
    if omega_m is None:
        omega_m     = 0.27             # Matter Density.
    omega_l     = 1.0 - omega_m
    c           = 2.99792458e5      # SPEED OF LIGHT    Units: km/s
    w0 = -1.31
    wp = 1.48   # p: prime symbol
    def Aint(z):
        return 1./np.sqrt( ((1.+z)**3)*omega_m + \
                         omega_l*((1.+z)**(3*(1+w0-wp)))*np.exp(3*wp*z) )
    AA      = integrate.quad(Aint, 0.0, z)
    DL_Mpc  = (c * (1. + z) / H_knot) * AA[0]
    DL_pc = DL_Mpc * (1E6) # convert Mpc to pc
    return DL_pc


def LumDist_weylgravity(redshift, H_knot=None):
    """
    Weyl Gravity.
    Equation 237 in Mannheim 2006 paper
    q_knot      = -0.37 or -0.2
    """
    z           = redshift
    if H_knot is None:
        H_knot      = 71.               # Hubble Constant.  Units: km/s/Mpc
    c           = 2.99792458e5      # SPEED OF LIGHT    Units: km/s
    q_knot      = -0.37
    DL_Mpc  = (-c * ((1. + z)**2) / (H_knot * q_knot)) * \
                (1-np.sqrt(1 + q_knot - (q_knot/((1+z)**2))))
    DL_pc = DL_Mpc * (1E6) # convert Mpc to pc
    return DL_pc

# def LumDist_wexpansion_2(redshift):
#     H_knot      = 71               # Hubble Constant.  Units: km/s/Mpc
#     omega_m     = 0.27             # Matter Density.
#     omega_l     = 1.0 - omega_m
#     c          = 2.99792458e5      # SPEED OF LIGHT    Units: km/s
#     w0 = -1.31
#     wa = 

#     z = redshift
#     def Aint(z):
#         return 1./np.sqrt( ((1.+z)**3)*omega_m + \
#                          omega_l*((1.+z)**(3*(1+w0+wa)))*np.exp(-3*wz*z/(1+z)) )
#     AA      = integrate.quad(Aint, 0.0, z)
#     DL_Mpc  = (c * (1. + z) / H_knot) * AA[0]
#     DL_pc = DL_Mpc * (1E6)
#     return DL_pc

    
def distance_modulus(lumdistance):
    return 5*np.log10(lumdistance)-5


def get_moduli(model, redshifts=None, H_knot=None, omega_m=None):
    """
    model : str. 'concord', 'weyl', or 'riess'
                    or 'c', 'w', and 'r'
    
    """
    H0 = H_knot
    OM = omega_m
    if model.startswith('c'):   
        DLs = [LumDist_concordance(redshift=z, H_knot=H0, omega_m=OM) for z in redshifts]
    elif model.startswith('r'):   
        DLs = [LumDist_riess(redshift=z, H_knot=H0, omega_m=OM) for z in redshifts]
    elif model.startswith('w'):   
        DLs = [LumDist_weylgravity(redshift=z, H_knot=H0) for z in redshifts]
    moduli = [distance_modulus(lumdistance=dL) for dL in DLs]
    return moduli

In [5]:
def calc_eiso(fluence, lumdist, redshift):
    flue = fluence
    DL = lumdist 
    z = redshift
    return flue*((4.0*pi*(DL**2))/(1.0+z))

In [6]:
mu = lambda values,errors: np.nansum([i/(j**2) for i,j in zip(values,errors)])/np.nansum([(j**-2.) for j in errors])
mu_unc = lambda errors: (np.nansum([(j**-2.) for j in errors]))**-0.5

Read Data
---

### 1 keV-10 keV Fluxes, Fluences, and Eisos:  Flux3, Flue3, Eiso3
---
The Eisos in this file were computed with our concordance cosmology.

The errors on flux and fluence in this file are **Margins of Error**

Margins of error can no be logged. Must find the Confidence Interval bounds first and then log those. 

Since we plant to calculate Eiso energies for these errors, we might as well convert them to confidence intervals immediately so that they are easier to work with. 

In [7]:
filename = ('/Users/kimzoldak/GRBs/sample/'
            'Band_13_GBM+LAT__19_GBMconstrained.txt')

df_ = pd.read_csv(filename)

In [8]:
df_.columns.tolist()

['trigger',
 'name',
 'number',
 'z',
 'LumDist',
 'trigtime',
 't90',
 't90Rest',
 't90start',
 't90stop',
 'detector',
 'function',
 'LATburst',
 'CSTAT',
 'LOGLIKE',
 'AIC',
 'BIC',
 'NDATA',
 'DOF',
 'flux',
 'flux_moe_lo',
 'flux_moe_up',
 'flux_ci_lo',
 'flux_ci_up',
 'fluence',
 'fluence_moe_lo',
 'fluence_moe_up',
 'fluence_ci_lo',
 'fluence_ci_up',
 'eiso',
 'eiso_moe_lo',
 'eiso_moe_up',
 'eiso_ci_lo',
 'eiso_ci_up',
 'epeak',
 'epeak_moe_lo',
 'epeak_moe_up',
 'epeak_ci_lo',
 'epeak_ci_up',
 'epeakRest',
 'epeakRest_moe_lo',
 'epeakRest_moe_up',
 'epeakRest_ci_lo',
 'epeakRest_ci_up',
 'alpha',
 'alpha_moe_lo',
 'alpha_moe_up',
 'alpha_ci_lo',
 'alpha_ci_up',
 'beta',
 'beta_moe_lo',
 'beta_moe_up',
 'beta_ci_lo',
 'beta_ci_up',
 'norm',
 'norm_moe_lo',
 'norm_moe_up',
 'norm_ci_lo',
 'norm_ci_up',
 'flux_10k10M',
 'flux_10k10M_moe_lo',
 'flux_10k10M_moe_up',
 'flux_10k10M_ci_lo',
 'flux_10k10M_ci_up',
 'flux_10k10G',
 'flux_10k10G_moe_lo',
 'flux_10k10G_moe_up',
 'flux_10k1

In [9]:
cols2keep = ['trigger', 'name', 'number', 'z', 't90', 'detector','LATburst'] 

# Make new dataframe with only certain columns
df = pd.DataFrame(df_.loc[:, cols2keep])

# Copy over flux and fluences from 1 keV-10 MeV. 

# Flux
df['flux'] = df_['flux_1k10M']
# confidence Intervals.
df['flux_ci_lo'] = df_['flux_1k10M_ci_lo'] 
df['flux_ci_up'] = df_['flux_1k10M_ci_up']

# Fluence
df['fluence'] = df_['fluence_1k10M']
# confidence Intervals.
df['fluence_ci_lo'] = df_['fluence_1k10M_ci_lo'] 
df['fluence_ci_up'] = df_['fluence_1k10M_ci_up']

In [10]:
df.loc[:, 'flux':'fluence_ci_up']

,flux,flux_ci_lo,flux_ci_up,fluence,fluence_ci_lo,fluence_ci_up
0,1.510120e-06,1.479451e-06,1.540789e-06,0.000095,0.000093,0.000097
1,1.138526e-06,1.113320e-06,1.163731e-06,0.000154,0.000150,0.000157
2,1.289260e-06,1.252148e-06,1.326373e-06,0.000080,0.000077,0.000082
3,1.289040e-05,1.199399e-05,1.378682e-05,0.000012,0.000012,0.000013
4,2.046451e-05,2.030564e-05,2.062338e-05,0.000396,0.000392,0.000399
5,1.297451e-05,1.290484e-05,1.304419e-05,0.000179,0.000178,0.000179
6,2.026665e-06,1.968604e-06,2.084725e-06,0.000041,0.000040,0.000042
7,6.484519e-07,5.840776e-07,7.128262e-07,0.000008,0.000007,0.000009
8,8.423418e-07,8.144667e-07,8.702168e-07,0.000139,0.000135,0.000144
9,4.409283e-06,4.335022e-06,4.483544e-06,0.000033,0.000032,0.000034


Now our errors on flux and fluence are **Confidence Intervals.**

These can now be logged and can be used for computing confidence intervals on Eiso.

In [11]:
#   The luminosity distances of these functions are in parsecs. Need to use pctocm to convert to cm.
# 1 pc = 3.0856775814914E+18 cm

Mpctocm = 3.08567758E24
pctocm = 3.08567758E18

In [12]:
H_knot = 67.8
omega_m = 0.308

redshifts = df.z.values

In [13]:
# Concordance Cosmology (Lambda CDM)
df['DL1'] = np.asarray([LumDist_concordance(redshift=z, 
                                            H_knot=H_knot, 
                                            omega_m=omega_m) for z in redshifts])*pctocm

# Riess Cosmology
df['DL2'] = np.asarray([LumDist_riess(redshift=z, 
                                      H_knot=H_knot, 
                                      omega_m=omega_m) for z in redshifts])*pctocm

# Weyl Gravity
df['DL3'] = np.asarray([LumDist_weylgravity(redshift=z, 
                                            H_knot=H_knot) for z in redshifts])*pctocm


### In the code below, we will compute values and confidence intervals, but we WANT margins of error.  
### Eiso errors are symmetric, so we don't need to compute both moe_lo and moe_up

    moe_lo = val - ci_lo
    moe_up = ci_up - val

In [14]:
# Concordance Cosmology (Lambda CDM)
df['eiso1']         = calc_eiso(fluence=df.fluence, lumdist=df['DL1'], redshift=df.z)
df['eiso1_ci_lo']   = calc_eiso(fluence=df.fluence_ci_lo, lumdist=df['DL1'], redshift=df.z)
df['eiso1_ci_up']   = calc_eiso(fluence=df.fluence_ci_up, lumdist=df['DL1'], redshift=df.z)
df['eiso1_err']     = df['eiso1']-df['eiso1_ci_lo']  # MOE. Eiso errs should be symmetrical when unlogged

# Riess Cosmology
df['eiso2']         = calc_eiso(fluence=df.fluence, lumdist=df['DL2'], redshift=df.z)
df['eiso2_ci_lo']   = calc_eiso(fluence=df.fluence_ci_lo, lumdist=df['DL2'], redshift=df.z)
df['eiso2_ci_up']   = calc_eiso(fluence=df.fluence_ci_up, lumdist=df['DL2'], redshift=df.z)
df['eiso2_err']     = df['eiso2']-df['eiso2_ci_lo']  # MOE. Eiso errs should be symmetrical when unlogged


# Weyl Gravity
df['eiso3']         = calc_eiso(fluence=df.fluence, lumdist=df['DL3'], redshift=df.z)
df['eiso3_ci_lo']   = calc_eiso(fluence=df.fluence_ci_lo, lumdist=df['DL3'], redshift=df.z)
df['eiso3_ci_up']   = calc_eiso(fluence=df.fluence_ci_up, lumdist=df['DL3'], redshift=df.z)
df['eiso3_err']     = df['eiso3']-df['eiso3_ci_lo']  # margin of error. Eiso errs should be symmetrical when unlogged



In [17]:
df.eiso1.head()

0    3.494756e+54
1    4.143551e+54
2    1.189908e+53
3    2.789666e+52
4    3.390065e+54
Name: eiso1, dtype: float64

In [15]:
df.head()

,trigger,name,number,z,t90,detector,LATburst,flux,flux_ci_lo,flux_ci_up,...,eiso1_ci_up,eiso1_err,eiso2,eiso2_ci_lo,eiso2_ci_up,eiso2_err,eiso3,eiso3_ci_lo,eiso3_ci_up,eiso3_err
0,bn080916009,GRB080916C,5,4.350,62.977,L,Y,0.000002,0.000001,0.000002,...,3.565731e+54,7.097540e+52,1.903003e+54,1.864354e+54,1.941651e+54,3.864830e+52,9.773098e+54,9.574615e+54,9.971581e+54,1.984830e+53
1,bn090323002,GRB090323,13,3.570,135.170,L,Y,0.000001,0.000001,0.000001,...,4.235283e+54,9.173245e+52,2.584131e+54,2.526922e+54,2.641341e+54,5.720908e+52,9.562792e+54,9.351085e+54,9.774498e+54,2.117069e+53
2,bn090328401,GRB090328A,14,0.736,61.697,L,Y,0.000001,0.000001,0.000001,...,1.224161e+53,3.425265e+51,1.197090e+53,1.162631e+53,1.231550e+53,3.445940e+51,1.246110e+53,1.210240e+53,1.281981e+53,3.587048e+51
3,bn090510016,GRB090510,17,0.903,0.960,L,Y,0.000013,0.000012,0.000014,...,2.983663e+52,1.939968e+51,2.751065e+52,2.559753e+52,2.942378e+52,1.913124e+51,3.035709e+52,2.824603e+52,3.246816e+52,2.111069e+51
4,bn090902462,GRB090902B,20,1.822,19.328,L,Y,0.000020,0.000020,0.000021,...,3.416383e+54,2.631759e+52,2.943145e+54,2.920296e+54,2.965993e+54,2.284808e+52,4.787187e+54,4.750023e+54,4.824350e+54,3.716366e+52


In [17]:
df.to_csv('eisoenergies_diff_cosmo.txt', sep=',', index=False)

Matplotlib's plt.errorbar command only takes margins of error. We must convert our confidence intervals to margins of error. We have lower and upper errors, so we make them a (2,N) array, where N is the number of GRBs. 

In [ ]:
axLims = (51.51, 55.2)


plt.clf()
#plt.figure(figsize=(8,7))
plt.figure()

# alpha=0.25

xaxis = 'eiso1'
yaxis = 'eiso2'
pltKwgs = dict(fmt='o', color='white', ecolor='blue', ms=5, lw=0.6, mec='blue', mew=0.75,
               capsize=0, alpha=1, label='x: LCDM\ny: Riess')
x = df[xaxis].apply(np.log10)
y = df[yaxis].apply(np.log10)
xerr = np.asarray([(df[xaxis].apply(np.log10)-df[xaxis+'_ci_lo'].apply(np.log10)).values, # lower margin of error
                   (df[xaxis+'_ci_up'].apply(np.log10)-df[xaxis].apply(np.log10)).values]) # upper margin of error
yerr = np.asarray([(df[yaxis].apply(np.log10)-df[yaxis+'_ci_lo'].apply(np.log10)).values, # lower margin of error
                   (df[yaxis+'_ci_up'].apply(np.log10)-df[yaxis].apply(np.log10)).values]) # upper margin of error
#plt.errorbar(x=x, y=y, yerr=yerr, xerr=xerr, **pltKwgs)
markers, caps, bars = plt.errorbar(x=x, y=y, yerr=yerr, xerr=xerr, **pltKwgs)
[bar.set_alpha(0.25) for bar in bars]
[cap.set_alpha(0.25) for cap in caps]


xaxis = 'eiso1'
yaxis = 'eiso3'
pltKwgs = dict(fmt='o', color='white', ecolor='red', ms=5, lw=0.6, mec='red', mew=0.75,
               capsize=0, alpha=1, label='x: LCDM\ny: Weyl')
x = df[xaxis].apply(np.log10)
y = df[yaxis].apply(np.log10)
xerr = np.asarray([(df[xaxis].apply(np.log10)-df[xaxis+'_ci_lo'].apply(np.log10)).values, # lower margin of error
                   (df[xaxis+'_ci_up'].apply(np.log10)-df[xaxis].apply(np.log10)).values]) # upper margin of error
yerr = np.asarray([(df[yaxis].apply(np.log10)-df[yaxis+'_ci_lo'].apply(np.log10)).values, # lower margin of error
                   (df[yaxis+'_ci_up'].apply(np.log10)-df[yaxis].apply(np.log10)).values]) # upper margin of error
#plt.errorbar(x=x, y=y, yerr=yerr, xerr=xerr, **pltKwgs)
markers, caps, bars = plt.errorbar(x=x, y=y, yerr=yerr, xerr=xerr, **pltKwgs)
[bar.set_alpha(0.25) for bar in bars]
[cap.set_alpha(0.25) for cap in caps]
    

xaxis = 'eiso2'
yaxis = 'eiso3'
pltKwgs = dict(fmt='o', color='white', ecolor='green', ms=5, lw=0.6, mec='green', mew=0.75,
               capsize=0, alpha=1, label='x: Riess\ny: Weyl')
x = df[xaxis].apply(np.log10)
y = df[yaxis].apply(np.log10)
xerr = np.asarray([(df[xaxis].apply(np.log10)-df[xaxis+'_ci_lo'].apply(np.log10)).values, # lower margin of error
                   (df[xaxis+'_ci_up'].apply(np.log10)-df[xaxis].apply(np.log10)).values]) # upper margin of error
yerr = np.asarray([(df[yaxis].apply(np.log10)-df[yaxis+'_ci_lo'].apply(np.log10)).values, # lower margin of error
                   (df[yaxis+'_ci_up'].apply(np.log10)-df[yaxis].apply(np.log10)).values]) # upper margin of error
#plt.errorbar(x=x, y=y, yerr=yerr, xerr=xerr, **pltKwgs)
markers, caps, bars = plt.errorbar(x=x, y=y, yerr=yerr, xerr=xerr, **pltKwgs)
[bar.set_alpha(0.25) for bar in bars]
[cap.set_alpha(0.25) for cap in caps]
#[marker.set_alpha(0.25) for marker in markers]


# Plot a line at equal values of x and y 
xlinedata = np.linspace(axLims[0], axLims[1], 10)
ylinedata = 1.0*xlinedata+0
plt.plot(xlinedata, ylinedata, 'k-', lw=1, alpha=0.25) 

plt.xlim(*axLims)
plt.ylim(*axLims)
plt.legend(loc=0, fontsize=11, labelspacing=0.7, handletextpad=0, frameon=False)
plt.xlabel('$E_{iso}$ (erg)', fontsize=13)
plt.ylabel('$E_{iso}$ (erg)', fontsize=13)
#plt.show()

filename = ('compareEiso__diff_Cosmo.pdf')
plt.savefig(filename)




In [ ]:
os.system('open %s'%filename)

In [ ]:
pwd

In [ ]:
!open ./

In [ ]:
# Put all your data into other lists
x_list = [x1, x2, x3]
y_list = [y1, y2, y3]
err_list = [err1, err2, err3]
formats = ['ro', 'bs', 'g^']

# Loop through data and plot
for x, y, err, f in zip(x_list, y_list, err_list, formats):
    markers, caps, bars = plt.errorbar(x, y, xerr=err, yerr=err, fmt=f, ms=10)
    [bar.set_alpha(0.5) for bar in bars]
    [cap.set_alpha(0.5) for cap in caps]

plt.show()

Eisos are larger when LCDM model is used, relative to Reiss. 

Eisos are larger when Weyl Gravity is used, relative to LCDM.

Eisos are larger when Weyl Gravity is used, relative to Reiss.

Weyl > LCDM > Reiss

This matches the Hubble Diagram. When Eisos are larger, so are their distance modulus magnitudes. On that plot we see Weyl > LCDM > Reiss as well. 

Therefore, based on the fact that H0=65 is the top of the shadded regions for each comological model curve and H0=75 is the bottom limit to the shaded regions, we expect smaller H0 to return higher Eiso and larger H0 to return lower Eiso. 

Latex Table
---

In [18]:
colnames = ['trigger', 'number', 'z', 'DL1', 'DL2', 'DL3'] 

# create new dataframe for printing to latex table.
df2 = pd.DataFrame(df.loc[:, colnames])


# Luminosity Distances
df2['DL1'] = df.loc[:, 'DL1']/1.E28
df2['DL2'] = df.loc[:, 'DL2']/1.E28
df2['DL3'] = df.loc[:, 'DL3']/1.E28

# Eiso
for p in ['eiso1', 'eiso2', 'eiso3']:
    value    = df[p]
    MOE_low  = df[p] - df[p+'_ci_lo']    # lower margin of error
    MOE_up   = df[p+'_ci_up'] - df[p]    # upper margin of error
    MOE      = np.asarray([np.mean([i,j]) for i,j in zip(MOE_low, MOE_up)]) # average the two.
    df2[p]  = value/1.E53
    df2[p+'_err'] = MOE/1.E53
    del p, value, MOE_low, MOE_up, MOE

In [19]:
df2.head()

,trigger,number,z,DL1,DL2,DL3,eiso1,eiso1_err,eiso2,eiso2_err,eiso3,eiso3_err
0,bn080916009,5,4.350,12.507880,9.229856,20.916615,34.947560,0.709754,19.030025,0.386483,97.730979,1.984830
1,bn090323002,13,3.570,9.895276,7.814458,15.032594,41.435509,0.917325,25.841315,0.572091,95.627916,2.117069
2,bn090328401,14,0.736,1.437554,1.441886,1.471112,1.189908,0.034253,1.197090,0.034459,1.246110,0.035870
3,bn090510016,17,0.903,1.847659,1.834831,1.927417,0.278967,0.019400,0.275107,0.019131,0.303571,0.021111
4,bn090902462,20,1.822,4.387160,4.087759,5.213383,33.900655,0.263176,29.431445,0.228481,47.871865,0.371637


In [20]:
for d in df2.values:
    print('%s & %i & %.2f & %.2f & %.2f & %.2f & %.2f$\pm$%.2f & %.2f$\pm$%.2f & %.2f$\pm$%.2f \\\\'%(
            d[0], d[1], d[2], d[3], d[4], d[5], d[6], d[7], d[8], d[9], d[10], d[11]))

bn080916009 & 5 & 4.35 & 12.51 & 9.23 & 20.92 & 34.95$\pm$0.71 & 19.03$\pm$0.39 & 97.73$\pm$1.98 \\
bn090323002 & 13 & 3.57 & 9.90 & 7.81 & 15.03 & 41.44$\pm$0.92 & 25.84$\pm$0.57 & 95.63$\pm$2.12 \\
bn090328401 & 14 & 0.74 & 1.44 & 1.44 & 1.47 & 1.19$\pm$0.03 & 1.20$\pm$0.03 & 1.25$\pm$0.04 \\
bn090510016 & 17 & 0.90 & 1.85 & 1.83 & 1.93 & 0.28$\pm$0.02 & 0.28$\pm$0.02 & 0.30$\pm$0.02 \\
bn090902462 & 20 & 1.82 & 4.39 & 4.09 & 5.21 & 33.90$\pm$0.26 & 29.43$\pm$0.23 & 47.87$\pm$0.37 \\
bn090926181 & 21 & 2.11 & 5.24 & 4.77 & 6.49 & 19.81$\pm$0.11 & 16.41$\pm$0.09 & 30.44$\pm$0.16 \\
bn091003191 & 24 & 0.90 & 1.83 & 1.82 & 1.91 & 0.91$\pm$0.03 & 0.90$\pm$0.03 & 0.99$\pm$0.03 \\
bn091208410 & 28 & 1.06 & 2.26 & 2.22 & 2.41 & 0.25$\pm$0.02 & 0.24$\pm$0.02 & 0.29$\pm$0.03 \\
bn100728095 & 33 & 1.57 & 3.65 & 3.46 & 4.17 & 9.06$\pm$0.30 & 8.18$\pm$0.27 & 11.86$\pm$0.39 \\
bn110731465 & 42 & 2.83 & 7.49 & 6.37 & 10.31 & 6.08$\pm$0.10 & 4.39$\pm$0.07 & 11.50$\pm$0.19 \\
bn130518580 & 58 & 2.49

# 1: LCDM
# 2: Riess
# 3: Weyl


### Always do: $ ∆ = y_{axis} - x_{axis}$
### When the data points trend below the diagonal, the values on the x-axis are larger than those on the y-axis, and a strong negative ∆ value results.

In [21]:
from Zoldak.math.tools import root_mean_square as rms

In [20]:
sigmas = []

a = 'eiso1'  # LCDM   x-axis
b = 'eiso2'  # Riess  y-axis
deltas = df[b].apply(np.log10) - df[a].apply(np.log10)  #   y-axis - x-axis
print('%.3f'%deltas.mean())
sigmas.append(rms(deltas))
print('%.3f'%rms(deltas))
print('')

a = 'eiso1'  # LCDM   x-axis
b = 'eiso3'  # Weyl   y-axis
deltas = df[b].apply(np.log10) - df[a].apply(np.log10)
print('%.3f'%deltas.mean())
sigmas.append(rms(deltas))
print('%.3f'%rms(deltas))
print('')

a = 'eiso2'  # Riess  x-axis
b = 'eiso3'  # Weyl   y-axis
deltas = df[b].apply(np.log10) - df[a].apply(np.log10)
print('%.3f'%deltas.mean())
sigmas.append(rms(deltas))
print('%.3f'%rms(deltas))
print('')



print(['%.3f'%i for i in sigmas])
print('')
print('mean: %.3f'%np.mean(sigmas))

-0.062
0.096

0.137
0.184

0.199
0.280

['0.096', '0.184', '0.280']

mean: 0.187


$\Delta_{avg}$:  -0.062, 0.137, and 0.199

$\sigma_{rms}$: 0.096, 0.184, and 0.280  with a mean of 0.187


In [29]:
def statistical_uncertainty(dataframe, parameter, suffix='', logit=False):
    """
    dataframe : pandas dataframe. Finds the values and confidence intervals 
                    within the dataframe. 

    parameter : str, string name of the parameter. 
                e.g., 'alpha', 'eiso', ...

    suffix : str, default is ''. If the parameter name and its errors has a 
                suffix of '_x' or '_y' from joining dataframes, place that 
                in the suffix. When we merge dataframes, our '_x' and '_y' 
                are after the '_moe_lo', so adding '_x' at the end of the 
                parameter string wont work.

    logit : True or False, default is False. 
            logit = True will apply log10 to the data and confidence intervals.
    
    Prints:
    -------
    Float. The sample's statistical uncertainty, rounded to 3 decimal places.
    
    Returns:
    -------
    Float. Same as prints, but does not round.
    
    Math:
    -----
    Averages the upper and lower confidence intervals (in their margin of 
    error form) for each GRB in the sample and then takes the average of the 
    all of those to give the *sample* average. 
    
    
    Example:
    --------
    for par,logit in zip(['eiso', 'alpha', 'beta', 'epeak', 'norm'],
                         [True, False, False, True, True]):
        statistical_uncertainty(dataframe=df1, parameter=par, logit=logit)
    
    """
    # only a single dataframe
    d = dataframe
    
    if logit == True:
        values = d[parameter+'%s'%suffix].apply(np.log10)
        ci_lo = d[parameter+'_ci_lo'+'%s'%suffix].apply(np.log10)
        ci_up = d[parameter+'_ci_up'+'%s'%suffix].apply(np.log10)
    else:
        values = d[parameter+'%s'%suffix]
        ci_lo = d[parameter+'_ci_lo'+'%s'%suffix]
        ci_up = d[parameter+'_ci_up'+'%s'%suffix] 
    a = np.mean([np.mean([i,j]) for i,j in zip(values-ci_lo, ci_up-values)])
    print('%.3f'%a)
    return a


In [35]:
statistical_uncertainty(dataframe=df, parameter='eiso1', suffix='', logit=True)
statistical_uncertainty(dataframe=df, parameter='eiso2', suffix='', logit=True)
statistical_uncertainty(dataframe=df, parameter='eiso3', suffix='', logit=True)

0.018
0.018
0.018


0.01800747473938047

In [37]:
np.sqrt(0.018**2 + 0.083**2)

0.08492938243034621

In [25]:
df.eiso1_err.apply(np.log10).mean()

51.79544196847853

In [28]:
df.eiso1_err.apply(np.log10)#mean()

0     52.851108
1     52.962523
2     51.534694
3     51.287795
4     52.420246
5     52.026912
6     51.416910
7     51.397777
8     52.477060
9     52.010148
10    52.434227
11    51.995408
12    51.229777
13    51.798716
14    52.154831
15    50.985129
16    51.572866
17    51.367447
18    50.442606
19    51.681317
20    52.177629
21    50.769983
22    52.299574
23    51.700493
24    51.330657
25    52.177542
26    52.383183
27    52.307136
28    51.799591
29    50.887598
30    52.153817
31    51.419444
Name: eiso1_err, dtype: float64

In [23]:
sigmas = []

a = 'eiso1'
b = 'eiso2'
deltas = df1[a].apply(np.log10) - df1[b].apply(np.log10)
sigmas.append(rms(deltas))
#print('%.3f'%rms(deltas))

a = 'eiso1'
b = 'eiso3'
deltas = df1[a].apply(np.log10) - df1[b].apply(np.log10)
sigmas.append(rms(deltas))
#print('%.4f'%rms(deltas))

a = 'eiso2'
b = 'eiso3'
deltas = df1[a].apply(np.log10) - df1[b].apply(np.log10)
sigmas.append(rms(deltas))
#print('%.3f'%rms(deltas))

print(['%.3f'%i for i in sigmas])
print('')
print('mean: %.3f'%np.mean(sigmas))

['0.096', '0.184', '0.280']

mean: 0.187


In [24]:
sigmas

[0.09648448195749751, 0.18446980959054327, 0.28016250608226595]

The `eiso_err_low` and `eiso_err_up` we have here are confidence intervals of `eiso`. They are symmetric in linear space. If we want to present the data as value +- err, then the errors must be margins of error. On top of that, if we wish to present the data in logged space, then we have a few steps to take and considerations to make. 

- `[MOE_low, MOE_up] = [value - CI_low, CI_up - value]`  should be identical because they are symmetrix in linear space. 

- Symmetry is not preserved under a linear to log transformation, therefore our lower and upper margin on error will be asymmetrical in log space. If we want to present them as a single value, we will have to average the two, after we properly handle them. 

- What we mean by properly handle them:  Margins of error can NOT simply be logged. You can only log values and their confidence interval bounds. `[logged_MOE_low, logged_MOE_up] = [log10(value)-log10(CI_low), log10(CI_up)-log10(value)]`.  Then take `np.mean([logged_MOE_low, logged_MOE_up])`

When using pandas dataframes you will have an array of data that must match. So taking `np.mean([logged_MOE_low, logged_MOE_up])` will be incorrect if `logged_MOE_low` and `logged_MOE_up` are arrays of data. Must do `err = [np.mean([i,j]) for i,j in zip(logged_MOE_low, logged_MOE_up)]`.